https://medium.com/towards-data-science/unsupervised-nlp-topic-models-as-a-supervised-learning-input-cf8ee9e5cf28
https://github.com/marcmuon/nlp_yelp_review_unsupervised/blob/master/notebooks/2-train_corpus_prep_and_LDA_train.ipynb


In [5]:
import pickle
import os
import gensim
import pyLDAvis
# import pyLDAvis.gensim
import spacy
import pandas as pd
import nltk; nltk.download('stopwords')
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import re
import warnings
from pprint import pprint
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import seaborn as sns
%config InlineBackend.figure_formats = ['retina']
from sklearn.metrics import f1_score
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import KFold
from sklearn.metrics import fbeta_score
import matplotlib.pyplot as plt

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\phili\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:
directory_path = r"C:\Users\phili\techlabs\nlp_delft\annotated_csv"

# Get a list of all files and directories in the specified directory
all_files = os.listdir(directory_path)

# Filter out directories and keep only files
files_only = [file for file in all_files if os.path.isfile(os.path.join(directory_path, file))]

# Sort files based on their modification time
sorted_files = sorted(files_only, key=lambda x: os.path.getmtime(os.path.join(directory_path, x)))

# Make list
dataframe = []
# Get the last 6 files for test dataset and the rest for training
training_files = sorted_files[0:15]
test_files = sorted_files[-6:]

for filename in training_files:
    file_path = os.path.join(directory_path, filename)
    df = pd.read_csv(file_path)
    dataframe.append(df)

train_df = pd.concat(dataframe, ignore_index=True)

dataframe = []

for filename in test_files:
    file_path = os.path.join(directory_path, filename)
    df = pd.read_csv(file_path)
    dataframe.append(df)


test_df = pd.concat(dataframe, ignore_index=True)

c:\Users\phili\anaconda3\envs\nlp_venv\Lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])


In [25]:
train_df.head(5)

,Unnamed: 0,0,Class
0,0,Article\nProcess Development for Methyl Isobut...,0.0
1,1,", Raed R. Alkathiri 1 and Abdulaziz A. Bagabas 2",0.0
2,2,"1 Chemical Engineering Department, King Saud U...",0.0
3,3,"Technology (KACST), P.O. Box 6086, Riyadh 1144...",0.0
4,4,* Correspondence: arabiah@ksu.edu.sa; Tel.: +9...,0.0


In [26]:
test_df.head(5)

,Unnamed: 0,0,Class
0,0,Review\nA Review on Deactivation and Regenerat...,0
1,1,"Joanna Sobczak, Izabela Wysocka",0
2,2,", Stanisław Murgrabia and Andrzej Rogala *",0
3,3,Department of Process Engineering and Chemical...,0
4,4,Abstract: The deactivation of catalysts and th...,0


In [27]:
train_df = train_df.rename(columns={"0": "Paragraph"})
test_df = test_df.rename(columns={"0": "Paragraph"})

In [28]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['come','order','try','go','get','make','drink','plate','dish','restaurant','place',
                  'would','really','like','great','service','came','got'])

In [29]:
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

In [30]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
        
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

In [31]:
def bigrams(words, bi_min=15, tri_min=10):
    bigram = gensim.models.Phrases(words, min_count = bi_min)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    return bigram_mod

In [32]:
def get_corpus(df):
    """
    Get Bigram Model, Corpus, id2word mapping
    """
    
    # df['text'] = strip_newline(df.text)
    words = list(sent_to_words(df.Paragraph))
    words = remove_stopwords(words)
    bigram = bigrams(words)
    bigram = [bigram[review] for review in words]
#     lemma = lemmatization(bigram)
    id2word = gensim.corpora.Dictionary(bigram)
    id2word.filter_extremes(no_below=10, no_above=0.35)
    id2word.compactify()
    corpus = [id2word.doc2bow(text) for text in bigram]
    return corpus, id2word, bigram

In [37]:
train_corpus4, train_id2word4, bigram_train4 = get_corpus(train_df)

In [38]:

bigram_train4[0]

['article',
 'process',
 'development',
 'methyl_isobutyl',
 'ketone',
 'production',
 'using',
 'low_pressure',
 'one_step',
 'gas_phase',
 'selective',
 'hydrogenation',
 'acetone',
 'abdulrahman',
 'al',
 'rabiah']

In [39]:
import logging
logging.basicConfig(filename='lda_model.log', format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    lda_train4 = gensim.models.ldamulticore.LdaMulticore(
                           corpus=train_corpus4,
                           num_topics=20,
                           id2word=train_id2word4,
                           chunksize=100,
                           workers=7, # Num. Processing Cores - 1
                           passes=50,
                           eval_every = 1,
                           per_word_topics=True)
    lda_train4.save('lda_train4.model')

In [40]:
coherence_model_lda = CoherenceModel(model=lda_train4, texts=bigram_train4, dictionary=train_id2word4, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
coherence_lda

0.507721050891222

In [41]:
lda_train4.print_topics(20,num_words=15)[:10]

[(0,
  '0.102*"catalyst" + 0.087*"catalysts" + 0.067*"al" + 0.052*"dimethyl_ether" + 0.051*"synthesis" + 0.031*"conversion" + 0.030*"formation" + 0.028*"energies" + 0.025*"selectivity" + 0.025*"direct" + 0.022*"dehydration" + 0.019*"syngas" + 0.017*"zeolite" + 0.017*"et_al" + 0.017*"reaction"'),
 (1,
  '0.034*"model" + 0.033*"used" + 0.031*"et_al" + 0.028*"case" + 0.026*"parameters" + 0.025*"using" + 0.025*"data" + 0.025*"activity" + 0.023*"pem" + 0.023*"copper" + 0.022*"based" + 0.022*"system" + 0.019*"operating" + 0.018*"method" + 0.017*"study"'),
 (2,
  '0.086*"reactor" + 0.052*"temperature" + 0.042*"steam" + 0.034*"feed" + 0.032*"gas" + 0.025*"bar" + 0.024*"stream" + 0.024*"pressure" + 0.023*"water" + 0.023*"air" + 0.022*"waste" + 0.015*"gases" + 0.015*"heat_exchanger" + 0.014*"combustion" + 0.012*"outlet"'),
 (3,
  '0.145*"processes" + 0.069*"extraction" + 0.036*"table" + 0.030*"resources" + 0.028*"step" + 0.028*"preparation" + 0.025*"composition" + 0.023*"puriﬁcation" + 0.020*"so

In [44]:
train_vecs = []
for i in range(len(train_df)):
    top_topics = lda_train4.get_document_topics(train_corpus4[i], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(20)]
    # topic_vec.extend([train_df.iloc[i].real_counts]) # counts of reviews for restaurant
    topic_vec.extend([len(train_df.iloc[i].Paragraph)]) # length review
    train_vecs.append(topic_vec)

c:\Users\phili\anaconda3\envs\nlp_venv\Lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
c:\Users\phili\anaconda3\envs\nlp_venv\Lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])
c:\Users\phili\anaconda3\envs\nlp_venv\Lib\site-packages\pandas\core\dtypes\cast.py:1641: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Depr

In [45]:
train_vecs[2]

[0.0071429336,
 0.0071429336,
 0.0071429336,
 0.0071429336,
 0.0071429336,
 0.0071429336,
 0.0071429336,
 0.8642843,
 0.0071429336,
 0.0071429336,
 0.0071429336,
 0.0071429336,
 0.0071429336,
 0.0071429336,
 0.0071429336,
 0.0071429336,
 0.0071429336,
 0.0071429336,
 0.0071429336,
 0.0071429336,
 184]

In [46]:
X = np.array(train_vecs)

In [47]:
y = np.array(train_df.Class)

In [52]:
y = np.nan_to_num(y)

In [53]:
with open('y.pkl', 'wb') as f:
    pickle.dump(y, f)

In [54]:
with open('X.pkl', 'wb') as f:
    pickle.dump(X, f)

In [57]:
y[y == 11] = 1

In [58]:
np.unique(y)

array([0., 1.])

In [59]:
kf = KFold(5, shuffle=True, random_state=42)
cv_lr_f1, cv_lrsgd_f1, cv_svcsgd_f1,  = [], [], []

for train_ind, val_ind in kf.split(X, y):
    # Assign CV IDX
    X_train, y_train = X[train_ind], y[train_ind]
    X_val, y_val = X[val_ind], y[val_ind]
    
    # Scale Data
    scaler = StandardScaler()
    X_train_scale = scaler.fit_transform(X_train)
    X_val_scale = scaler.transform(X_val)

    # Logisitic Regression
    lr = LogisticRegression(
        class_weight= 'balanced',
        solver='newton-cg',
        fit_intercept=True
    ).fit(X_train_scale, y_train)

    y_pred = lr.predict(X_val_scale)
    cv_lr_f1.append(f1_score(y_val, y_pred, average='binary'))
    
    # Logistic Regression Mini-Batch SGD
    sgd = linear_model.SGDClassifier(
        max_iter=1000,
        tol=1e-3,
        loss='log',
        class_weight='balanced'
    ).fit(X_train_scale, y_train)
    
    y_pred = sgd.predict(X_val_scale)
    cv_lrsgd_f1.append(f1_score(y_val, y_pred, average='binary'))
    
   # SGD Modified Huber
    sgd_huber = linear_model.SGDClassifier(
        max_iter=1000,
        tol=1e-3,
        alpha=20,
        loss='modified_huber',
        class_weight='balanced'
    ).fit(X_train_scale, y_train)
    
    y_pred = sgd_huber.predict(X_val_scale)
    cv_svcsgd_f1.append(f1_score(y_val, y_pred, average='binary'))

print(f'Logistic Regression Val f1: {np.mean(cv_lr_f1):.3f} +- {np.std(cv_lr_f1):.3f}')
print(f'Logisitic Regression SGD Val f1: {np.mean(cv_lrsgd_f1):.3f} +- {np.std(cv_lrsgd_f1):.3f}')
print(f'SVM Huber Val f1: {np.mean(cv_svcsgd_f1):.3f} +- {np.std(cv_svcsgd_f1):.3f}')


c:\Users\phili\anaconda3\envs\nlp_venv\Lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(
c:\Users\phili\anaconda3\envs\nlp_venv\Lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(
c:\Users\phili\anaconda3\envs\nlp_venv\Lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


Logistic Regression Val f1: 0.222 +- 0.033
Logisitic Regression SGD Val f1: 0.175 +- 0.032
SVM Huber Val f1: 0.173 +- 0.100


c:\Users\phili\anaconda3\envs\nlp_venv\Lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(
c:\Users\phili\anaconda3\envs\nlp_venv\Lib\site-packages\sklearn\linear_model\_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


In [60]:

from sklearn.model_selection import KFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Define hyperparameters and their potential values for grid search
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize a Random Forest classifier
rf_model = RandomForestClassifier(random_state=42)

# Define the number of folds for k-fold cross-validation
n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

# Initialize GridSearchCV with k-fold cross-validation
grid_search = GridSearchCV(rf_model, param_grid, cv=kf, scoring='accuracy')

# Perform grid search on the entire dataset
grid_search.fit(X, y)

# Get the best parameters and best estimator from grid search
best_params = grid_search.best_params_
best_rf_model = grid_search.best_estimator_

# Print the best parameters
print("Best Parameters:", best_params)


Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}


In [63]:
cv_rf_f1 = []
y_pred = best_rf_model.predict(X_val_scale)
cv_rf_f1.append(f1_score(y_val, y_pred, average='binary'))
print(f'Random Forest Val f1: {np.mean(cv_rf_f1):.3f} +- {np.std(cv_rf_f1):.3f}')


Random Forest Val f1: 0.246 +- 0.000
